In [27]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Concatenate, Dot, Lambda, Input
from keras.datasets import mnist
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
import pickle
from sklearn import svm
from keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from google.colab.patches import cv2_imshow

In [28]:
!pwd
#!rm -rf Gestures

/content


In [29]:
!git clone https://github.com/KuldeepDileep/Gestures.git

fatal: destination path 'Gestures' already exists and is not an empty directory.


In [30]:
import zipfile
with zipfile.ZipFile('/content/Gestures/Gestures-Images.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Gestures')

In [31]:
TrainDir = '/content/Gestures/Gestures-Images/ASL_Images'

count=0
current_id = 0
label_ids = {}
x_train = []
y_labels= []

image_train = []
label_train =[]


#---------------------------------------Extracting Train Images-----------------------------------------
for root, dirs, files in os.walk(TrainDir):
    for file in files:
        if file.endswith("png") or  file.endswith("jpeg") or file.endswith("jpg"):
            count+=1
            path = os.path.join(root, file)
            label = os.path.basename(os.path.dirname(path)).replace(" ",",").lower()
            label = os.path.basename(root).replace(" ",",").lower()
            if not label in label_ids: #if the current label is not in dictionsry we save it with the corresponding current_id i.e 0,1,2
                label_ids[label] = current_id
                current_id +=1
            id_ = label_ids[label]
            label_train.append(id_)

            
            pil_image =  Image.open(path) #Opening the image using Image object and converting it into Gray
            #pil_image = pil_image.resize((224,224))
            #pil_image = pil_image.convert("RGB")
            image_array =  np.array(pil_image, "uint8") #converting the image into a numpy array
            image_train.append(image_array)
            #print(image_array)
print("----------------------------Label_ids----------------------------------")
print(label_ids)
print(label_ids)
print(count)

----------------------------Label_ids----------------------------------
{'name': 0, 'my': 1, 'hello': 2}
{'name': 0, 'my': 1, 'hello': 2}
600


In [32]:
image_train = np.asarray(image_train)
label_train = np.asarray(label_train)
x_train = image_train
y_train = label_train
print(y_train.shape)
print(x_train.shape)

(600,)
(600, 215, 240)


Spliting data:

In [33]:
#-----------------------------------------------Test and Train Spli--------------------------------------
x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.2)
print(x_train.shape)
print(x_test.shape)

(480, 215, 240)
(120, 215, 240)


Data Normalization:

In [34]:
x_train = x_train/np.max(x_train)
x_test = x_test/np.max(x_test)

In [35]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(480, 215, 240)
(480,)
(120, 215, 240)
(120,)


RBF Kernel SVM:

In [36]:
x_train = x_train.reshape((x_train.shape[0], 215*240))
x_test = x_test.reshape((x_test.shape[0], 215*240))
rbf_svc = svm.SVC(kernel='rbf', C=2000, gamma=.009)
rbf_svc.fit(x_train, y_train)
rbf_acc = rbf_svc.predict(x_test)
print("Accuracy: "+str(accuracy_score(y_test, rbf_acc)))

Accuracy: 0.9833333333333333


In [37]:
filename = 'kernelsvm_model.sav'
pickle.dump(rbf_svc, open(filename, 'wb'))
print("Saved model to disk")

Saved model to disk


In [38]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(x_test, y_test)
print("Testing accuracy: ", result)

Testing accuracy:  0.9833333333333333
